In [1]:
from riptide import *

In [6]:
iCdJ794 = cobra.io.read_sbml_model('../data/iCdJ794.sbml')

In [4]:
iCdJ794

Name,iCdJ794
Memory address,0x07f213c96b410
Number of metabolites,1134
Number of reactions,1129
Objective expression,0.0 + 1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular"


In [30]:
print('Genes: ' + str(len(list(iCdJ794.genes))))

Genes: 794


In [23]:
exch = 0
for rxn in iCdJ794.reactions:
    if len(list(rxn.products)) == 0:
        exch += 1
print('Exchanges: ' + str(exch))

Exchanges: 99


In [24]:
trans = 0
for rxn in iCdJ794.reactions:
    comps = set([x.compartment for x in list(rxn.reactants)] + [x.compartment for x in list(rxn.products)])
    if len(comps) > 1:
        trans += 1
print('Transporters: ' + str(trans))

Transporters: 104


In [25]:
metab = len(list(iCdJ794.reactions)) - exch - trans
print('Metabolic reactions: ' + str(metab))

Metabolic reactions: 936


In [29]:
no_gpr, freemass, unbalanced, no_trans, no_exch = checkQuality(iCdJ794)

Model is overdetermined
102 metabolic reactions not associated with genes
54 reactions are charge imbalanced


In [20]:
blocked, nogene_blocked, nogene_gapfilled_blocked = blockedReactions(iCdJ794)

485 total reactions are blocked
14 reactions without GPRs are blocked
0 gapfilled reactions without GPRs are blocked


In [2]:
# Define media conditions
mdm = ['cpd00001_e', # water
       'cpd00065_e', # L-Tryptophan
       'cpd00060_e', # L-Methionine
       'cpd00322_e', # L-Isoleucine
       'cpd00129_e', # L-Proline
       'cpd00156_e', # L-Valine
       'cpd00107_e', # L-Leucine
       'cpd00084_e', # L-Cysteine 
       'cpd00149_e', # Cobalt
       'cpd00099_e', # Chloride
       'cpd10515_e', # Iron
       'cpd00030_e', # Manganese
       'cpd00254_e', # Magnesium
       'cpd00063_e', # Calcium
       'cpd00205_e', # Potassium
       'cpd00009_e', # Phosphate
       'cpd00971_e', # Sodium
       'cpd00242_e', # Carbonate
       'cpd00104_e', # Biotin
       'cpd00305_e', # Thiamine
       'cpd00263_e', # Pyridoxine
       'cpd00027_e'] # D-Glucose (Carbohydrate C-source)

# Theriot et al. (2013). Nature Communications.
ncmm = ['cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine 
        'cpd00065_e', # L-Tryptophan
        'cpd00060_e', # L-Methionine
        'cpd00119_e', # L-Histidine
        'cpd00033_e', # Glycine
        'cpd00051_e', # L-Arginine
        'cpd00161_e'] # L-Threonine


In [3]:
# check gene essentiality
def essentialGenes(model):
    essential = set()
    for gene in model.genes:
        with model as m:
            old_bounds = []
            for rxn in gene.reactions:
                old_bounds.append(rxn.bounds)
                rxn.bounds = (0.0, 0.0)
            
            obj_val = m.slim_optimize()
            if obj_val < 1e-6: essential |= set([gene.id])
            
            index = 0
            for rxn in gene.reactions:
                rxn.bounds = old_bounds[index]
                index += 1
                
    print('Essential genes: ' + str(len(essential)))
    return essential


In [7]:
complete_essential_single = essentialGenes(iCdJ794)

Essential genes: 78


### RIPTiDe *in vivo* Contextualization

In [8]:
# Read in formatted transcription files
def read_transcription(infile):
    
    abund_dict = {}
    with open(infile, 'r') as abunds:
        header = abunds.readline()
        for line in abunds:
            line = line.split()
            gene = line[0].split('|')[0]
            abund = float(line[2])
            
            abund_dict[gene] = abund
            
    return abund_dict


In [9]:
# Read in in vivo C. difficile transcription
cefoperazone = read_transcription('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/transcript/cefoperazone_630.mapped.norm.tsv')
clindamycin = read_transcription('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/transcript/clindamycin_630.mapped.norm.tsv')
streptomycin = read_transcription('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/transcript/streptomycin_630.mapped.norm.tsv')
gnotobiotic = read_transcription('/home/mjenior/Desktop/repos/Jenior_iCdJ794_2019/data/transcript/gnotobiotic_630.mapped.norm.tsv')

In [24]:
iCdJ794_cef, cef_samples = riptide(iCdJ794, cefoperazone)


Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific flux distributions (longest step)...

Reactions pruned to 291 from 1129 (74.22% reduction)
Metabolites pruned to 289 from 1134 (74.51% reduction)
Flux through the objective REDUCED to ~76.48 from ~89.77 (14.8% shift)
Solution space volume DECREASED to ~1785.892 from ~8460.514 (78.89% shift)

RIPTiDe completed in 3.0 minutes and 37.0 seconds



In [25]:
cef_essential_genes = essentialGenes(iCdJ794_cef)

Essential genes: 151


In [12]:
iCdJ794_clinda, clinda_samples = riptide(iCdJ794, clindamycin)


Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific flux distributions (longest step)...

Reactions pruned to 283 from 1129 (74.93% reduction)
Metabolites pruned to 280 from 1134 (75.31% reduction)
Flux through the objective REDUCED to ~75.09 from ~89.77 (16.35% shift)
Solution space volume DECREASED to ~1147.782 from ~8460.514 (86.43% shift)

RIPTiDe completed in 3.0 minutes and 1.0 seconds



In [13]:
clinda_essential_genes = essentialGenes(iCdJ794_clinda)

Essential genes: 147


In [14]:
iCdJ794_strep, strep_samples = riptide(iCdJ794, streptomycin)


Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific flux distributions (longest step)...

Reactions pruned to 288 from 1129 (74.49% reduction)
Metabolites pruned to 285 from 1134 (74.87% reduction)
Flux through the objective REDUCED to ~80.25 from ~89.77 (10.6% shift)
Solution space volume DECREASED to ~1334.615 from ~8460.514 (84.23% shift)

RIPTiDe completed in 3.0 minutes and 25.0 seconds



In [15]:
strep_essential_genes = essentialGenes(iCdJ794_strep)

Essential genes: 147


In [16]:
iCdJ794_gnoto, gnoto_samples = riptide(iCdJ794, gnotobiotic)


Initializing model and parsing transcriptome...
Pruning zero flux subnetworks...
Sampling context-specific flux distributions (longest step)...

Reactions pruned to 281 from 1129 (75.11% reduction)
Metabolites pruned to 281 from 1134 (75.22% reduction)
Flux through the objective REDUCED to ~73.13 from ~89.77 (18.54% shift)
Solution space volume DECREASED to ~890.108 from ~8460.514 (89.48% shift)

RIPTiDe completed in 4.0 minutes and 6.0 seconds



In [17]:
gnoto_essential_genes = essentialGenes(iCdJ794_gnoto)

Essential genes: 157


In [18]:
# Compares lists to create diagrams for 4 groups
def venn_comparison(list1, list2, list3, list4):
        
    # Confirm correct data types
    list1 = set(list1)
    list2 = set(list2)
    list3 = set(list3)
    list4 = set(list4)
    
    # Identify exclusive elements
    list1_only = list1.difference(list2)
    list1_only = list1_only.difference(list3)
    list1_only = list1_only.difference(list4)
    list2_only = list2.difference(list1)
    list2_only = list2_only.difference(list3)
    list2_only = list2_only.difference(list4)
    list3_only = list3.difference(list1)
    list3_only = list3_only.difference(list2)
    list3_only = list3_only.difference(list4)
    list4_only = list4.difference(list1)
    list4_only = list4_only.difference(list2)
    list4_only = list4_only.difference(list3)

    # Find overlap between just 2 groups
    list1_list2_overlap = list1.intersection(list2)
    list1_list2_overlap = list1_list2_overlap.difference(list3)
    list1_list2_overlap = list1_list2_overlap.difference(list4)
    list1_list3_overlap = list1.intersection(list3)
    list1_list3_overlap = list1_list3_overlap.difference(list2)
    list1_list3_overlap = list1_list3_overlap.difference(list4)
    list1_list4_overlap = list1.intersection(list4)
    list1_list4_overlap = list1_list4_overlap.difference(list2)
    list1_list4_overlap = list1_list4_overlap.difference(list3)
    list2_list3_overlap = list2.intersection(list3)
    list2_list3_overlap = list2_list3_overlap.difference(list1)
    list2_list3_overlap = list2_list3_overlap.difference(list4)
    list2_list4_overlap = list2.intersection(list4)
    list2_list4_overlap = list2_list4_overlap.difference(list1)
    list2_list4_overlap = list2_list4_overlap.difference(list3)
    list3_list4_overlap = list3.intersection(list4)
    list3_list4_overlap = list3_list4_overlap.difference(list1)
    list3_list4_overlap = list3_list4_overlap.difference(list2)

    # Find overlap in 3 groups
    list1_list2_list3_overlap = list1.intersection(list2)
    list1_list2_list3_overlap = list1_list2_list3_overlap.intersection(list3)
    list1_list2_list3_overlap = list1_list2_list3_overlap.difference(list4)
    list1_list2_list4_overlap = list1.intersection(list2)
    list1_list2_list4_overlap = list1_list2_list4_overlap.intersection(list4)
    list1_list2_list4_overlap = list1_list2_list4_overlap.difference(list3)
    list1_list3_list4_overlap = list1.intersection(list3)
    list1_list3_list4_overlap = list1_list3_list4_overlap.intersection(list4)
    list1_list3_list4_overlap = list1_list3_list4_overlap.difference(list2)
    list2_list3_list4_overlap = list2.intersection(list3)
    list2_list3_list4_overlap = list2_list3_list4_overlap.intersection(list4)
    list2_list3_list4_overlap = list2_list3_list4_overlap.difference(list1)
    
    # Find overlap between all groups
    all_list_overlap = list1.intersection(list2)
    all_list_overlap = all_list_overlap.intersection(list3)
    all_list_overlap = all_list_overlap.intersection(list4)
    
    # Calculate totals in each group
    list1_total = float(len(list1))
    list2_total = float(len(list2))
    list3_total = float(len(list3))
    list4_total = float(len(list4))
    list1_only_total = float(len(list1_only))
    list2_only_total = float(len(list2_only))
    list3_only_total = float(len(list3_only))
    list4_only_total = float(len(list4_only))
    list1_list2_overlap_total = float(len(list1_list2_overlap))
    list1_list3_overlap_total = float(len(list1_list3_overlap))
    list1_list4_overlap_total = float(len(list1_list4_overlap))
    list2_list3_overlap_total = float(len(list2_list3_overlap))
    list2_list4_overlap_total = float(len(list2_list4_overlap))
    list3_list4_overlap_total = float(len(list3_list4_overlap))
    list1_list2_list3_overlap_total = float(len(list1_list2_list3_overlap))
    list1_list2_list4_overlap_total = float(len(list1_list2_list4_overlap))
    list1_list3_list4_overlap_total = float(len(list1_list3_list4_overlap))
    list2_list3_list4_overlap_total = float(len(list2_list3_list4_overlap))
    all_list_overlap_total = float(len(all_list_overlap))
    
    # Calculate percent overlaps
    list1_only_percent = round(((list1_only_total / list1_total) * 100.0), 1)
    list2_only_percent = round(((list2_only_total / list2_total) * 100.0), 1)
    list3_only_percent = round(((list3_only_total / list3_total) * 100.0), 1)
    list4_only_percent = round(((list4_only_total / list4_total) * 100.0), 1)
    temp1 = (list1_list2_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_overlap_total / list2_total) * 100.0
    list1_list2_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list3_overlap_total / list1_total) * 100.0
    temp2 = (list1_list3_overlap_total / list3_total) * 100.0
    list1_list3_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list4_overlap_total / list4_total) * 100.0
    list1_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list2_list3_overlap_total / list2_total) * 100.0
    temp2 = (list2_list3_overlap_total / list3_total) * 100.0
    list2_list3_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list2_list4_overlap_total / list2_total) * 100.0
    temp2 = (list2_list4_overlap_total / list4_total) * 100.0
    list2_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list3_list4_overlap_total / list3_total) * 100.0
    temp2 = (list3_list4_overlap_total / list4_total) * 100.0
    list3_list4_overlap_percent = round(numpy.mean([temp1, temp2]), 1)
    temp1 = (list1_list2_list3_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_list3_overlap_total / list2_total) * 100.0
    temp3 = (list1_list2_list3_overlap_total / list3_total) * 100.0
    list1_list2_list3_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list1_list2_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list2_list4_overlap_total / list2_total) * 100.0
    temp3 = (list1_list2_list4_overlap_total / list4_total) * 100.0
    list1_list2_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list1_list3_list4_overlap_total / list1_total) * 100.0
    temp2 = (list1_list3_list4_overlap_total / list3_total) * 100.0
    temp3 = (list1_list3_list4_overlap_total / list4_total) * 100.0
    list1_list3_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (list2_list3_list4_overlap_total / list2_total) * 100.0
    temp2 = (list2_list3_list4_overlap_total / list3_total) * 100.0
    temp3 = (list2_list3_list4_overlap_total / list4_total) * 100.0
    list2_list3_list4_overlap_percent = round(numpy.mean([temp1, temp2, temp3]), 1)
    temp1 = (all_list_overlap_total / list1_total) * 100.0
    temp2 = (all_list_overlap_total / list2_total) * 100.0
    temp3 = (all_list_overlap_total / list3_total) * 100.0
    temp4 = (all_list_overlap_total / list4_total) * 100.0
    all_list_overlap_percent = round(numpy.mean([temp1, temp2, temp3, temp4]), 1)
    
    # Print report to the screen
    print('List 1 only: ' + str(list1_only_percent) + '% (' + str(int(list1_only_total)) + ')')
    print('List 2 only: ' + str(list2_only_percent) + '% (' + str(int(list2_only_total)) + ')')
    print('List 3 only: ' + str(list3_only_percent) + '% (' + str(int(list3_only_total)) + ')')
    print('List 4 only: ' + str(list4_only_percent) + '% (' + str(int(list4_only_total)) + ')')
    print('')
    print('List 1 + List 2: ' + str(list1_list2_overlap_percent) + '% (' + str(int(list1_list2_overlap_total)) + ')')
    print('List 1 + List 3: ' + str(list1_list3_overlap_percent) + '% (' + str(int(list1_list3_overlap_total)) + ')')
    print('List 1 + List 4: ' + str(list1_list4_overlap_percent) + '% (' + str(int(list1_list4_overlap_total)) + ')')
    print('List 2 + List 3: ' + str(list2_list3_overlap_percent) + '% (' + str(int(list2_list3_overlap_total)) + ')')
    print('List 2 + List 4: ' + str(list2_list4_overlap_percent) + '% (' + str(int(list2_list4_overlap_total)) + ')')
    print('List 3 + List 4: ' + str(list3_list4_overlap_percent) + '% (' + str(int(list3_list4_overlap_total)) + ')')
    print('')
    print('List 1 + List 2 + List 3: ' + str(list1_list2_list3_overlap_percent) + '% (' + str(int(list1_list2_list3_overlap_total)) + ')')
    print('List 1 + List 2 + List 4: ' + str(list1_list2_list4_overlap_percent) + '% (' + str(int(list1_list2_list4_overlap_total)) + ')')
    print('List 1 + List 3 + List 4: ' + str(list1_list3_list4_overlap_percent) + '% (' + str(int(list1_list3_list4_overlap_total)) + ')')
    print('List 2 + List 3 + List 4: ' + str(list2_list3_list4_overlap_percent) + '% (' + str(int(list2_list3_list4_overlap_total)) + ')')
    print('')
    print('Shared: ' + str(all_list_overlap_percent) + '% (' + str(int(all_list_overlap_total)) + ')')

    # Return new lists
    return [list1_only,list2_only,list3_only,list4_only,list1_list2_overlap, list1_list3_overlap, list1_list4_overlap, list2_list3_overlap, list2_list4_overlap, list3_list4_overlap, list1_list2_list3_overlap, list1_list2_list4_overlap, list1_list3_list4_overlap, list2_list3_list4_overlap, all_list_overlap]


In [19]:
# Context specific gene essentiality
# 1. cefoperazone
# 2. clindamycin
# 3. streptomycin
# 4. gnotobiotic
essentiality = venn_comparison(cef_essential_genes, clinda_essential_genes, strep_essential_genes, gnoto_essential_genes)

List 1 only: 3.3% (5)
List 2 only: 0.7% (1)
List 3 only: 0.7% (1)
List 4 only: 8.3% (13)

List 1 + List 2: 0.0% (0)
List 1 + List 3: 0.0% (0)
List 1 + List 4: 2.6% (4)
List 2 + List 3: 0.7% (1)
List 2 + List 4: 0.7% (1)
List 3 + List 4: 0.0% (0)

List 1 + List 2 + List 3: 4.0% (6)
List 1 + List 2 + List 4: 0.0% (0)
List 1 + List 3 + List 4: 0.7% (1)
List 2 + List 3 + List 4: 2.0% (3)

Shared: 89.8% (135)


In [18]:
# Gnotobiotic only
essentiality[3]

{'272563.8.peg.1421',
 '272563.8.peg.1786',
 '272563.8.peg.1810',
 '272563.8.peg.2321',
 '272563.8.peg.3244',
 '272563.8.peg.3321',
 '272563.8.peg.3439',
 '272563.8.peg.3683',
 '272563.8.peg.570',
 '272563.8.peg.757',
 '272563.8.peg.921',
 'CD630_24170',
 'CD630_24180'}

In [19]:
# Conventional only
essentiality[10]

{'272563.8.peg.2840',
 '272563.8.peg.2865',
 '272563.8.peg.3390',
 '272563.8.peg.3438',
 '272563.8.peg.3656',
 'CD630_06820'}

In [20]:
# Cefoperazone only
essentiality[0]

{'272563.8.peg.1040',
 '272563.8.peg.2022',
 '272563.8.peg.2278',
 '272563.8.peg.871',
 '272563.8.peg.872'}

In [21]:
# Cefoperazone & Gnotobiotic only
essentiality[6]

{'272563.8.peg.105',
 '272563.8.peg.1574',
 '272563.8.peg.2713',
 '272563.8.peg.866'}

In [26]:
iCdJ794_gnoto.reactions[1].metabolites

{<Metabolite cpd00011_c at 0x7f59299deb10>: 1.0,
 <Metabolite cpd00061_c at 0x7f59299fac50>: 1.0,
 <Metabolite cpd00032_c at 0x7f59299fadd0>: -1.0,
 <Metabolite cpd00068_c at 0x7f59299fae50>: -1.0,
 <Metabolite cpd00090_c at 0x7f59299faf50>: 1.0}

### Context-specific substrate importance

In [65]:
def test_growth_substrates(genre, extracellular):
    
    obj_val = genre.slim_optimize()
    
    for test in range(1,100):
        important = set()
        with genre as g:
            for rxn in g.reactions:
                if len(rxn.reactants) == 0 or len(rxn.products) == 0: 
                    rxn.upper_bound = 1
                    if g.slim_optimize() < obj_val: 
                        current = []
                        for cpd in rxn.metabolites:
                            if cpd.compartment == extracellular:
                                important |= set([cpd.id + ' --- ' + cpd.name])                    
                    rxn.upper_bound = 1000
    
    print('Important growth substrates: ' + str(len(important)))
    return important
    

In [66]:
cef_substrates = test_growth_substrates(iCdJ794_cef, 'extracellular')

Important growth substrates: 56


In [60]:
clinda_substrates = test_growth_substrates(iCdJ794_clinda, 'extracellular')

Important growth substrates: 13


In [61]:
strep_substrates = test_growth_substrates(iCdJ794_strep, 'extracellular')

Important growth substrates: 15


In [62]:
gnoto_substrates = test_growth_substrates(iCdJ794_gnoto, 'extracellular')

Important growth substrates: 16


In [45]:
# Context specific growth substrates
# 1. cefoperazone
# 2. clindamycin
# 3. streptomycin
# 4. gnotobiotic
substrates = venn_comparison(cef_substrates, clinda_substrates, strep_substrates, gnoto_substrates)

List 1 only: 18.6% (11)
List 2 only: 4.3% (2)
List 3 only: 0.0% (0)
List 4 only: 6.3% (1)

List 1 + List 2: 42.0% (22)
List 1 + List 3: 11.7% (4)
List 1 + List 4: 0.0% (0)
List 2 + List 3: 3.1% (1)
List 2 + List 4: 0.0% (0)
List 3 + List 4: 0.0% (0)

List 1 + List 2 + List 3: 18.6% (7)
List 1 + List 2 + List 4: 10.1% (3)
List 1 + List 3 + List 4: 0.0% (0)
List 2 + List 3 + List 4: 0.0% (0)

Shared: 42.7% (12)


In [49]:
# Cefoperazone only
substrates[0]

{'cpd00036_e --- Succinate',
 'cpd00063_e --- Ca2+',
 'cpd00065_e --- L-Tryptophan',
 'cpd00092_e --- Uracil',
 'cpd00099_e --- Cl-',
 'cpd00104_e --- BIOT',
 'cpd00149_e --- Co2+',
 'cpd00158_e --- CELB',
 'cpd00205_e --- K+',
 'cpd00305_e --- Thiamin',
 'cpd03170_e --- 4-Hydroxymandelate'}

In [48]:
# Gnotobiotic only
substrates[3]

{'cpd00159_e --- L-Lactate'}

In [46]:
# Conventional only
substrates[10]

{'cpd00067_e --- H+',
 'cpd00076_e --- Sucrose',
 'cpd00080_e --- Glycerol-3-phosphate',
 'cpd00082_e --- D-Fructose',
 'cpd00307_e --- Cytosine',
 'cpd00443_e --- ABEE',
 'cpd00971_e --- Na+'}

In [50]:
# Core
substrates[-1]

{'cpd00011_e --- CO2',
 'cpd00013_e --- Ammonia',
 'cpd00029_e --- Acetate',
 'cpd00041_e --- L-Aspartate',
 'cpd00117_e --- D-Alanine',
 'cpd00122_e --- N-Acetyl-D-glucosamine',
 'cpd00141_e --- Propionate',
 'cpd00314_e --- D-Mannitol',
 'cpd00339_e --- 5-Aminopentanoate',
 'cpd00492_e --- N-Acetyl-D-mannosamine',
 'cpd00588_e --- Sorbitol',
 'cpd19585_e --- 2-Methylbutyrate'}